In [31]:

from modules.utils import load_datasets_statforecast_uni, blocked_rolling_origin_update


In [32]:
data_path = '/Users/moji/PyTSF-MfG/data'
datasets = load_datasets_statforecast_uni(data_path)

/Users/moji/PyTSF-MfG/data/AirPassengers.csv
(144, 3)
/Users/moji/PyTSF-MfG/data/ABM.csv
(10098, 3)


In [33]:
# Set parameters
lookback_window = 10
forecast_horizon = 3
rolling_step = 3
    
# Create train-test sets
train_test_sets = blocked_rolling_origin_update(datasets['air_passengers'], 
        target_column='y', 
        lookback_window=lookback_window, 
        forecast_horizon=forecast_horizon, 
        rolling_step=rolling_step
    )

In [34]:
# Print information about the splits
print(f'number of splits:{len(train_test_sets)}')
for i, (train_df, test_df) in enumerate(train_test_sets):
    if i >= 3:  # Limit to first 3 items
        break
    print(f"Split {i + 1}:")
    print(f"  Train period: {train_df.index[0]} to {train_df.index[-1]}")
    print(f"  Test period: {test_df.index[0]} to {test_df.index[-1]}")
    print(f"  Train samples: {len(train_df)}, Test samples: {len(test_df)}")
    print(train_df)
    print(test_df)




number of splits:44
Split 1:
  Train period: 0 to 9
  Test period: 10 to 12
  Train samples: 10, Test samples: 3
   unique_id         ds    y
0          1 1949-01-01  112
1          1 1949-02-01  118
2          1 1949-03-01  132
3          1 1949-04-01  129
4          1 1949-05-01  121
5          1 1949-06-01  135
6          1 1949-07-01  148
7          1 1949-08-01  148
8          1 1949-09-01  136
9          1 1949-10-01  119
    unique_id         ds    y
10          1 1949-11-01  104
11          1 1949-12-01  118
12          1 1950-01-01  115
Split 2:
  Train period: 3 to 12
  Test period: 13 to 15
  Train samples: 10, Test samples: 3
    unique_id         ds    y
3           1 1949-04-01  129
4           1 1949-05-01  121
5           1 1949-06-01  135
6           1 1949-07-01  148
7           1 1949-08-01  148
8           1 1949-09-01  136
9           1 1949-10-01  119
10          1 1949-11-01  104
11          1 1949-12-01  118
12          1 1950-01-01  115
    unique_id         ds

In [35]:
test_df['y'].values

array([170, 158, 133])

In [36]:
from statsforecast.models import AutoARIMA
model_params ={}
def build_model(params):
    return AutoARIMA(**params)

In [37]:
from modules.evaluator import Evaluator
from modules.results_saver import ResultsSaver
ALGORITHM = "AutoArima"
# Set parameters
lookback_window = 10
forecast_horizon = 3
rolling_step = 3

OUTPUT_DIR = '/Users/moji/PyTSF-MfG/results'
saver = ResultsSaver(OUTPUT_DIR)

for name, data in datasets.items():
    train_test_sets = blocked_rolling_origin_update(data, 
        target_column='y', 
        lookback_window=lookback_window, 
        forecast_horizon=forecast_horizon, 
        rolling_step=rolling_step
    )
    all_results = {}
    for i, (train_df, test_df) in enumerate(train_test_sets):    
        model = build_model(model_params)
        train_df = train_df.reset_index(drop=True)
        model.fit(train_df['y'])
        # print(f'model fitted for window {i})')
        prediction = model.predict(h=forecast_horizon)
        forecast = prediction['mean']
        # print(type(forecast))
        actual = test_df['y'].values
        iteration_metrics = Evaluator.calculate_metrics(actual, forecast)
        all_results[f"iteration_{i}"] = iteration_metrics
        
    final_results = Evaluator.aggregate_results(all_results)
    print(f"Final aggregated results for {name}:")
    for metric, value in final_results.items():
        print(f"{metric}: {value}")
    
    saver.save_results(final_results, ALGORITHM, name)
    print(f"Experiments completed for {ALGORITHM} on {name} and results saved.")

# save_results(results, 'autoarima_results.json')
print(" experiments completed. Results saved.")

Final aggregated results for air_passengers:
MSE: 2417.3701688556052
RSME: 40.473127123648624
MAE: 34.08438819520562
WAPE: 0.11275688728607071
MAPE: 0.11380926702883802
SMAPE: 11.407302425717292
RAE: 1.5120192515900621
RSE: 1.5726965963441835
MASE: 1.0569100797159063
R2: -1.7088677773642822
Experiments completed for AutoArima on air_passengers and results saved.
Final aggregated results for stock_prices:
MSE: 0.6148975888273088
RSME: 0.2985673475071124
MAE: 0.2691129240437465
WAPE: 0.018335297589351676
MAPE: 0.01835418734110978
SMAPE: 1.8708897477626578
RAE: 37121797.08974514
RSE: 219.3238303037673
MASE: 12373933.335832218
R2: -4911271.146455121
Experiments completed for AutoArima on stock_prices and results saved.
 experiments completed. Results saved.
